In [7]:
import gymnasium as gym
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tqdm import tqdm
import os

In [2]:
# Hyperparameters
learning_rate = 0.00025
epsilon = 0.1  # Exploration rate
episodes = 100  # Number of training episodes
optimizer = optimizers.Adam(learning_rate=learning_rate)
loss_function = 'mse'  # Mean Squared Error loss for Q-value difference


In [3]:
def create_cnn(input_shape, num_actions):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(64, (8, 8), strides=(4, 4), activation='relu'),
        layers.Conv2D(128, (4, 4), strides=(2, 2), activation='relu'),
        layers.Conv2D(256, (3, 3), strides=(2, 2), activation='relu'),  # First 256 layer
        layers.Conv2D(256, (3, 3), activation='relu'),  # Second 256 layer
        layers.Conv2D(256, (3, 3), activation='relu'),  # Third 256 layer
        layers.Conv2D(256, (1, 1), activation='relu'),  # Fourth 256 layer
        layers.Conv2D(256, (1, 1), activation='relu'),  # Fourth 256 layer
        layers.Conv2D(256, (1, 1), activation='relu'),  # Fourth 256 layer
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(5, activation='linear')  # Output layer for Q-values
    ])
    return model


In [4]:
env = gym.make('ALE/Frogger-v5', render_mode='rgb_array')
#env = gym.make('ALE/Frogger-v5', )
input_shape = env.observation_space.shape  # This should match the frame size
num_actions = env.action_space.n  # Number of possible actions

model = create_cnn(input_shape, num_actions)
model.compile(optimizer=optimizers.Adam(learning_rate=0.00025), loss='mse')  # Mean Squared Error loss for Q-value difference


In [ ]:
def train_model(model, episodes, i, save_interval=10, ):
    # Initialize list to keep track of total rewards for each episode
    episode_rewards = []

    # Create a directory to save model weights
    save_dir = "model_weights"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Set up tqdm progress bar
    with tqdm(total=episodes, desc="Episode", unit='episode') as pbar:
        for e in range(episodes):
            state = env.reset()[0]
            state = np.array(state)
            done = False
            total_reward = 0

            while not done:
                # Randomly choose an action or the best predicted action
                if np.random.rand() <= epsilon:  # Use the global epsilon value
                    action = env.action_space.sample()
                else:
                    q_values = model.predict(np.expand_dims(state, axis=0), verbose=0)
                    action = np.argmax(q_values[0])

                next_state, reward, terminated, truncated, info = env.step(action)
                next_state = np.array(next_state)
                total_reward += reward
                
                if terminated or truncated:
                    done = True

            # Update progress bar
            pbar.update(1)
            pbar.set_description(f"Episode: {e+1}, Reward: {total_reward}")

            # Append the total reward to the rewards list
            episode_rewards.append(total_reward)

            # Save the model every 'save_interval' episodes
            if (e + 1) % save_interval == 0:
                model_path = os.path.join(save_dir, f'model_episode_{i}{e + 1}.h5')
                model.save(model_path)
                print(f"Saved model at episode {e + 1} to {model_path}")

    # Print overall training results
    print(f"Average Reward: {np.mean(episode_rewards)}")
    print(f"Best Reward: {max(episode_rewards)}")

# Example usage
# train_model(model, 100)  # Adjust as needed for your setup
# env.close()

In [18]:
env = gym.make('ALE/Frogger-v5', render_mode='rgb_array')
input_shape = env.observation_space.shape  # This should match the frame size
num_actions = env.action_space.n  # Number of possible actions

model2 = create_cnn(input_shape, num_actions)
model2.load_weights('model_weights/model_episode_100.h5')
env.reset()
game_over = False
train_model(model2, 100, i=3)

Episode: 10, Reward: 8.0:  10%|█████                                             | 10/100 [02:56<25:47, 17.19s/episode]WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


Saved model at episode 10 to model_weights\model_episode_310.h5


Episode: 20, Reward: 8.0:  20%|██████████                                        | 20/100 [06:08<23:50, 17.88s/episode]WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


Saved model at episode 20 to model_weights\model_episode_320.h5


Episode: 30, Reward: 6.0:  30%|███████████████                                   | 30/100 [09:18<21:15, 18.22s/episode]WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


...

Saved model at episode 60 to model_weights\model_episode_360.h5


Episode: 70, Reward: 8.0:  70%|███████████████████████████████████               | 70/100 [20:26<07:26, 14.88s/episode]WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


Saved model at episode 70 to model_weights\model_episode_370.h5


Episode: 71, Reward: 11.0:  71%|██████████████████████████████████▊              | 71/100 [20:55<08:32, 17.69s/episode]


KeyboardInterrupt: 